In [ ]:
# default_exp utils

In [ ]:
import nbdev.showdoc as literacy

In [ ]:
#export
from speechsep.imports import *

# Utils

This contains helper functions used for spectrogram creation and audio processing.

## Fourier Transforms

In [ ]:
#export
def time_bins(X, window_size, overlap):
    """
    Create an overlapped version of X
    Parameters
    ----------
    X : ndarray, shape=(n_samples,)
        Input signal to window and overlap
    window_size : int
        Size of windows to take
    window_step : int
        Step size between windows
    Returns
    -------
    X_strided : shape=(nun_windows, window_size)
        2D array of overlapped X
    """
    if window_size % 2 != 0:
        raise ValueError(f"Window size must be even! Recieved {window_size}")
    padding = np.zeros(int(window_size - len(X) % window_size))
    X = np.concatenate((X, padding))
    slide_length = int(window_size*(1-overlap))
    num_windows = (len(X) - window_size) // slide_length
    out = np.ndarray((num_windows,window_size),dtype = X.dtype)

    for i in range(num_windows):
        start = i * slide_length
        out[i] = X[start : start+window_size]
    return out

In [ ]:
#export
def stft(X, fftsize=512, win_mult=2, overlap=0.5, normalize=False):
    """
    Compute STFT for 1D real valued input X
    """
    win_size = fftsize*win_mult
    X = time_bins(X, win_size, overlap)
    hanning = .54 - .46 * np.cos(2 * np.pi * np.arange(win_size) / (win_size - 1))
    X = X * hanning.reshape((1, win_size))
    X = np.fft.fft(X).T
    #X = np.fft.fft(X)[:win_size//2].T
    if normalize: X*=256/X.max()
    return X

In [ ]:
#export
def istft(X, fftsize=512, win_mult=2, overlap=0.5, normalize=False):
    #X = np.concatenate((X, X[::-1]), axis=0)
    X = np.fft.ifft(X.T).real
    win_size = len(X[0])
    slider_length = int(win_size*(1-overlap))

    hanning = .54 - .46 * np.cos(2 * np.pi * np.arange(win_size) / (win_size - 1))
    X = X/hanning.reshape((1, win_size))

    inv_audio = X[0][0:win_size-slider_length]
    for i in range(len(X)):
        inv_audio = np.concatenate((inv_audio, X[i][-slider_length:]))
    return(inv_audio)

## Transforms

In [ ]:
#export
def Resample(sr_new):
    def _inner(ai):
        '''Resample using faster polyphase technique and avoiding FFT computation'''
        if(ai.sr == sr_new): return AudioItem(ai)
        sig_np = ai.sig.numpy()
        sr_gcd = math.gcd(ai.sr, sr_new)
        resampled = resample_poly(sig_np, int(sr_new/sr_gcd), int(ai.sr/sr_gcd), axis=-1)
        resampled = resampled.astype(np.float32)
        return AudioItem((torch.from_numpy(resampled), sr_new, ai.path))
    return _inner

## Tests

## Documentation

In [ ]:
fn = "/home/holydemon/Music/LJ001-0001.wav"
audio,sr = load(fn)
display(Audio(audio, rate=sr))

In [ ]:
spec = stft(audio, win_mult=2, normalize=True)
audio_r = istft(spec, win_mult=2)
display(Audio(audio_r, rate=sr))